In [1]:
import numpy as np
import wave
import struct

# Bảng định nghĩa tần số DTMF: mỗi phím được biểu diễn bằng cặp tần số (low, high)
DTMF_FREQS = {
    '1': (697, 1209),
    '2': (697, 1336),
    '3': (697, 1477),
    'A': (697, 1633),
    '4': (770, 1209),
    '5': (770, 1336),
    '6': (770, 1477),
    'B': (770, 1633),
    '7': (852, 1209),
    '8': (852, 1336),
    '9': (852, 1477),
    'C': (852, 1633),
    '*': (941, 1209),
    '0': (941, 1336),
    '#': (941, 1477),
    'D': (941, 1633)
}

def generate_tone(key, duration=0.5, fs=8000):
    """
    Sinh ra tín hiệu DTMF cho một phím.
    - key: ký tự cần mã hóa.
    - duration: thời gian của tín hiệu (giây).
    - fs: tốc độ lấy mẫu (Hz).
    """
    low_freq, high_freq = DTMF_FREQS[key]
    t = np.linspace(0, duration, int(fs * duration), endpoint=False)
    # Tạo tín hiệu bằng cách cộng 2 sóng sin với tần số tương ứng
    tone = np.sin(2 * np.pi * low_freq * t) + np.sin(2 * np.pi * high_freq * t)
    # Chuẩn hóa tín hiệu về dải giá trị int16
    tone = tone * (32767 / np.max(np.abs(tone)))
    return tone.astype(np.int16)

def generate_dtmf_sequence(sequence, tone_duration=0.5, pause_duration=0.1, fs=8000):
    """
    Sinh ra tín hiệu DTMF cho chuỗi các ký tự.
    - sequence: chuỗi ký tự cần mã hóa.
    - tone_duration: thời gian phát của mỗi tín hiệu phím.
    - pause_duration: khoảng dừng giữa các tín hiệu.
    - fs: tốc độ lấy mẫu.
    """
    signal = np.array([], dtype=np.int16)
    pause = np.zeros(int(fs * pause_duration), dtype=np.int16)
    for key in sequence:
        tone = generate_tone(key, duration=tone_duration, fs=fs)
        signal = np.concatenate((signal, tone, pause))
    return signal

def save_wave(filename, signal, fs=8000):
    """
    Lưu tín hiệu âm thanh vào file .wav.
    """
    with wave.open(filename, 'w') as wf:
        wf.setnchannels(1)       # kênh mono
        wf.setsampwidth(2)       # 2 byte (16 bit)
        wf.setframerate(fs)
        wf.writeframes(signal.tobytes())

def decode_dtmf_from_signal(signal, fs=8000, tone_duration=0.5, pause_duration=0.1):
    """
    Giải mã chuỗi ký tự từ tín hiệu DTMF.
    Các bước:
      - Cắt tín hiệu thành các đoạn ứng với mỗi tone (dựa theo tone_duration và pause_duration).
      - Thực hiện FFT trên mỗi đoạn để tìm các tần số chiếm ưu thế.
      - So sánh với bảng DTMF để ánh xạ về ký tự ban đầu.
    """
    samples_per_tone = int(fs * tone_duration)
    samples_per_pause = int(fs * pause_duration)
    index = 0
    decoded = ""
    # Định nghĩa các tần số nhóm thấp và nhóm cao trong DTMF
    low_freqs = [697, 770, 852, 941]
    high_freqs = [1209, 1336, 1477, 1633]
    
    while index + samples_per_tone <= len(signal):
        chunk = signal[index:index + samples_per_tone]
        # Tính FFT của đoạn tín hiệu
        fft_vals = np.abs(np.fft.rfft(chunk))
        freqs = np.fft.rfftfreq(len(chunk), 1 / fs)
        # Lấy các chỉ số của các giá trị lớn nhất trong phổ
        peak_indices = np.argsort(fft_vals)[-10:]
        detected_low = None
        detected_high = None
        # Duyệt qua các đỉnh tìm ra tần số thuộc nhóm thấp và nhóm cao
        for idx in peak_indices:
            freq = freqs[idx]
            for lf in low_freqs:
                if abs(freq - lf) < 15:  # cho phép sai số khoảng 15 Hz
                    detected_low = lf
                    break
            for hf in high_freqs:
                if abs(freq - hf) < 15:
                    detected_high = hf
                    break
        # So sánh với bảng DTMF để tìm ký tự tương ứng
        for key, (lf, hf) in DTMF_FREQS.items():
            if detected_low == lf and detected_high == hf:
                decoded += key
                break
        # Di chuyển chỉ số đến đoạn tín hiệu tiếp theo (bỏ qua cả khoảng dừng)
        index += samples_per_tone + samples_per_pause
    return decoded

if __name__ == "__main__":
    # Ví dụ sử dụng: mã hóa chuỗi ký tự thành tín hiệu DTMF và lưu vào file .wav
    sequence = "123A456B789C*0#D"
    signal = generate_dtmf_sequence(sequence)
    save_wave("dtmf_tones.wav", signal)
    print("Đã lưu tín hiệu DTMF vào file dtmf_tones.wav")
    
    # Đọc file .wav và giải mã tín hiệu DTMF
    import scipy.io.wavfile as wav
    fs_read, signal_read = wav.read("dtmf_tones.wav")
    decoded_sequence = decode_dtmf_from_signal(signal_read, fs=fs_read)
    print("Chuỗi giải mã được:", decoded_sequence)


Đã lưu tín hiệu DTMF vào file dtmf_tones.wav
Chuỗi giải mã được: 123A456B789C*0#D
